explores data that scraper.py produced

In [66]:
import pandas as pd
import seaborn as sns

sns.set(style="whitegrid")

In [67]:
tweets = pd.read_csv("data/tweets.csv")
users = pd.read_csv("data/users.csv")

#### Tweets

In [68]:
print(len(tweets))
tweets[0:2]

454


,experiment_id,experiment_group,text,tweet_id,likes,retweets,created_at,user_id,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,screen_name
0,0,usedgov,@usedgov Thank you for loan forgiveness,1599146922790354944,1,0,Sat Dec 03 21:01:51 +0000 2022,1508197172080222214,1.597260e+18,20437286,usedgov,luzbaez628
1,1,usedgov,@usedgov @SecCardona @usesgov @potus #studentl...,1598411747349913601,0,0,Thu Dec 01 20:20:32 +0000 2022,1477370850147983367,1.595158e+18,20437286,usedgov,fabulosi_t


There are 454 datapoints split between the 3 sources (Dept of Education, CNN, Fox News). What are their frequency percentages?

In [69]:
pop = pd.DataFrame(
    tweets
        .groupby('in_reply_to_screen_name')
        .size()
        .reset_index(name="count")
)
pop.head()

,in_reply_to_screen_name,count
0,CNN,17
1,FoxNews,430
2,usedgov,7




in_reply_to_screen_name	count
0	CNN	17
1	FoxNews	430
2	usedgov	7

How many unique authors are there? 

In [70]:
pop = pd.DataFrame(
    tweets
        .groupby('screen_name')
        .size()
        .reset_index(name="count")
)

#user with more screen names
pop[pop['count'] >1]

,screen_name,count
54,Curious56388405,2
63,DahlmanCarl,2
68,DavidWo15416040,3
90,Forevertrue10,2
168,Marcos_Swagga,4
192,PCopposition,2
266,Vincent08571545,2
282,argo_times,6
321,fabulosi_t,2
418,thomaslew13,3


### Users

In [71]:
users[0:2]

,user_id,created_at,description,location,followers_count,screen_name,statuses_count,favourites_count,verified
0,1508197172080222214,Sun Mar 27 21:41:16 +0000 2022,Thoughtful,NaN,0,luzbaez628,31,3,False
1,1477370850147983367,Sat Jan 01 20:07:25 +0000 2022,Everything I do is done with fabulosiT.,NaN,74,fabulosi_t,5111,5524,False


In [72]:
users.columns

Index(['user_id', 'created_at', 'description', 'location', 'followers_count',
       'screen_name', 'statuses_count', 'favourites_count', 'verified'],
      dtype='object')